<div >
<img src = "../banner.jpg" />
</div>

# Spatial Data

In [ ]:
require("pacman")
p_load("tidyverse","sf","modeldata","geojsonio")

In [ ]:
data("ames", package = "modeldata")

In [ ]:
dim(ames)

In [ ]:
class(ames)

![](figs/mercator.gif)

In [ ]:
ames_sf <- sf::st_as_sf(
  ames,
  # "coords" is in x/y order -- so longitude goes first!
  coords = c("Longitude", "Latitude"),
  # Set our coordinate reference system to EPSG:4326,
  # the standard WGS84 geodetic coordinate reference system
  crs = 4326
)

In [ ]:
class(ames_sf)

In [ ]:
head(ames_sf)

In [ ]:
ggplot() +
    geom_sf(data=ames_sf)+
    theme_bw()

In [ ]:
p_load("leaflet")

In [ ]:
map1<-leaflet()  %>% 
        addTiles()  %>% 
        addCircleMarkers(data=ames_sf)

In [ ]:
#workaround to show in Jupyter Notebook (not needed in Rstudio)
p_load("htmlwidgets","IRdisplay")

saveWidget(map1, 'demo1.html', selfcontained = FALSE)
display_html('<iframe src="demo1.html" width="800" height="800"></iframe>')

In [ ]:
#Different Tiles
map2<-leaflet()  %>% 
    addProviderTiles(providers$Stamen.Toner)  %>% 
    addCircles(data=ames_sf)

In [ ]:
#workaround to show in Jupyter Notebook (not needed in Rstudio)
p_load("htmlwidgets","IRdisplay")

saveWidget(map2, 'demo2.html', selfcontained = FALSE)
display_html('<iframe src="demo2.html" width="800" height="800"></iframe>')

# Clustering
## Etapas

Las etapas del análisis de clusters podemos resumirlas de la siguiente forma:

1. Iniciamos con una matriz de datos

    \begin{align}
X_{n\times k}=\left(\begin{array}{cccc}
x_{11} &  & \dots & x_{1k}\\
\\
\vdots &  & x_{ik} & \vdots\\
\\
x_{n1} &  & \dots & x_{nk} 
\end{array}\right)
    \end{align}

2. Calculamos la matriz de distancia o disimilitud

\begin{align}
D_{n\times n}=\left(\begin{array}{ccccc}
d_{11} &  & \dots &  & d_{1n}\\
 & \ddots\\
\vdots &  & d_{jj} &  & \vdots\\
 &  &  & \ddots\\
d_{n1} &  & \dots &  & d_{nn}
\end{array}\right)
\end{align}


3. Aplicamos el algoritmo de clustering. Existen varios tipos, en esta semana nos centramos en aquellos  **Algoritmos basados en centroides**. En estos algoritmos cluster está representado por un centroide. Los clusters se construyen en función de la distancia al centroide del grupo.

In [ ]:
set.seed(101011)
ames_sample<-ames_sf  %>% sample_frac(size=1/3) 
db<- ames_sample  %>%  select(geometry)
head(db)

In [ ]:
db<-st_distance(db)
head(db)

In [ ]:
db<-units::drop_units(db)

In [ ]:
k3 <- kmeans(db, centers = 3, nstart = 25)
str(k3)

In [ ]:
ames_sample<- ames_sample %>% mutate(clusters=factor(k3$cluster))

In [ ]:
ggplot() +
  geom_sf(data=ames_sample,aes(col=clusters)) + #graficamos las predicciones
  theme_bw()


## Caveat

Los métodos de clustering son exploratorios: se pueden utilizar para evaluar la calidad de los datos y generar hipótesis. 

Pero no importa lo que entre en el algoritmo de agrupamiento, los clusters salen. Esta es una situación clásica de "basura que entra, basura que sale". 



![](figs/Garbage-Model.jp)



Obviamente, esperamos que lo que se está metiendo en el análisis no sea basura, pero eso no garantiza que salga una "pearl of wisdom".

La conclusión es que la agrupación es buena si es útil para responder el problema en particular. Pero, esto es difícil de evaluar.


Hay medidas de cuán bueno es el agrupamiento. Funcionan según el principio de que las distancias entre los elementos del mismo grupo deben ser pequeñas y las distancias entre los elementos de diferentes grupos deben ser grandes. 

Esta es una verificación interna de la "estrechez" de los grupos, pero no garantiza que los grupos sean útiles y/o significativos para el problema bajo estudio. Esto requiere que el usuario utilice su capacidad y discernimiento.

## ¿Cuántos K (clusters) debemos elegir?

### Método del codo

In [ ]:
# función que calcula la SSR within-cluster 
wss <- function(k) {
  kmeans(db, k, nstart = 25 )$tot.withinss
}

# Calculamos y graficamos para k = 1 hasta k = 12
wss_values <- sapply(1:12,wss)

plot(1:12, wss_values,
       type="b", pch = 19, frame = FALSE, 
       xlab="Número de clusters (K)",
       ylab="SSR within-clusters total")

### Coeficiente de Silhouette

In [ ]:
p_load("cluster")
# función para extraer el coeficiente de silhouette

avg_sil <- function(k) {
  km.res <- kmeans(db, centers = k, nstart = 25)
  ss <- cluster::silhouette(km.res$cluster, dist(db))
  mean(ss[, 3])
}


# Calcular el coeficiente de silhouette para  k = 2 hasta k = 12
valores_sil <-  sapply(2:12,avg_sil)

plot(2:12, valores_sil,
       type = "b", pch = 19, frame = FALSE, 
       xlab="Número de clusters (K)",
       ylab = "Coeficiente de Silhouette")

In [ ]:
k4 <- kmeans(db, centers = 4, nstart = 25)

ames_sample<- ames_sample %>% mutate(clusters=factor(k4$cluster))

In [ ]:
ggplot() +
  geom_sf(data=ames_sample,aes(col=clusters)) + #graficamos las predicciones
  theme_bw()

# Super learner

Vamos a modelar los precios de venta de las casas en el conjunto de datos de Ames. Digamos que el precio de venta de estas casas depende del año en que se construyeron, su superficie habitable (tamaño) y el tipo de casa que son (dúplex vs. townhouse vs. unifamiliar)

In [ ]:
ames<- ames  %>% mutate(logprice=log(Sale_Price))

In [ ]:
p_load("caret")
set.seed(1011)
inTrain <- createDataPartition(
  y = ames$logprice,## La variable dependiente u objetivo 
  p = .7, ## Usamos 70%  de los datos en el conjunto de entrenamiento 
  list = FALSE)


train <- ames[ inTrain,]
test  <- ames[-inTrain,]
colnames(train)

In [ ]:
p_load("SuperLearner")


In [ ]:
# Review available models.
listWrappers()

In [ ]:
ySL<-train$logprice
XSL<- train  %>% select(Year_Built, Bldg_Type, Gr_Liv_Area)

In [ ]:
sl.lib <- c("SL.randomForest", "SL.lm")

# Fit using the SuperLearner package,

fitY <- SuperLearner(Y = ySL,  X= data.frame(XSL),
    method = "method.NNLS", SL.library = sl.lib)

fitY

In [ ]:
test <- test  %>%  mutate(yhat_Sup=predict(fitY, newdata = data.frame(test), onlySL = T)$pred)
head(test$yhat_Sup)

In [ ]:
with(test,mean(abs(logprice-yhat_Sup)))

## Test algorithm with multiple hyperparameter settings

The performance of an algorithm varies based on its hyperparamters, which again are its configuration settings. Some algorithms may not vary much, and others might have far better or worse performance for certain settings. Often we focus our attention on 1 or 2 hyperparameters for a given algorithm because they are the most important ones.

For random forest there are two particularly important hyperparameters: mtry and maximum leaf nodes. Mtry is how many features are randomly chosen within each decision tree node - in other words, each time the tree considers making a split. Maximum leaf nodes controls how complex each tree can get.

Let's try 3 different mtry options.

In [ ]:
# Customize the defaults for random forest.
custon_ranger = create.Learner("SL.ranger", params = list(num.trees = 1000))

# Look at the object.
custon_ranger$names


In [ ]:
custom_rf = create.Learner("SL.randomForest",
                     tune = list(mtry = round(c(1, sqrt(3), 3))))
custom_rf$names

In [ ]:
# Customize the defaults for random forest.
custon_glmnet = create.Learner("SL.glmnet", tune = list(alpha = seq(0, 1, length.out=5)))

# Look at the object.
custon_glmnet$names

In [ ]:
sl.lib2 <- c("SL.randomForest", "SL.lm",custon_ranger$names,custon_glmnet$names,custom_rf$names)
sl.lib2

In [ ]:
# Fit (takes forever)

#fitY_long <- SuperLearner(Y = y, X = data.frame(X),
#     method = "method.NNLS", SL.library = sl.lib)

#fitY_long

# Spatial Cross Validation


In [ ]:
p_load("spatialsample")

ames_sf <- sf::st_as_sf(
  ames,
  # "coords" is in x/y order -- so longitude goes first!
  coords = c("Longitude", "Latitude"),
  # Set our coordinate reference system to EPSG:4326,
  # the standard WGS84 geodetic coordinate reference system
  crs = 4326
)


In [ ]:
set.seed(123)
block_folds <- spatial_block_cv(ames_sf, v = 15)



In [ ]:
autoplot(block_folds) + theme_bw()

In [ ]:

set.seed(123)
cluster_folds <- spatial_clustering_cv(ames_sf, v = 15)
autoplot(cluster_folds) + theme_bw()

In [ ]:
set.seed(123)
location_folds <- 
  spatial_leave_location_out_cv(
   ames_sf,
    group = Neighborhood,
    v = 15
  )

In [ ]:
autoplot(location_folds)+ theme_bw()

In [ ]:
table(ames_sf$Neighborhood)


In [ ]:
ames_sf <- ames_sf   %>% mutate(Neighborhood=droplevels(Neighborhood))

In [ ]:
table(ames_sf$Neighborhood)

In [ ]:
length(unique(ames_sf$Neighborhood))

In [ ]:
test_neigh<- ames_sf  %>% filter(Neighborhood=="North_Ames")
test_neigh <- test_neigh   %>% mutate(Neighborhood=droplevels(Neighborhood))
train_neigh<- ames_sf  %>% filter(Neighborhood!="North_Ames")
train_neigh <- train_neigh   %>% mutate(Neighborhood=droplevels(Neighborhood))

In [ ]:
y_neigh<-train_neigh$logprice
X_neigh<- train_neigh  %>% select(Year_Built, Bldg_Type, Gr_Liv_Area)  %>% st_drop_geometry()

In [ ]:
index <- split(1:nrow(train_neigh),train_neigh$Neighborhood)

In [ ]:
index

In [ ]:
folds<-length(index)
folds

In [ ]:
fitY_neigh <- SuperLearner(Y = y_neigh, X = data.frame(X_neigh),
    method = "method.NNLS", SL.library = sl.lib,
    cvControl = list(V = folds, validRows = index))

In [ ]:
fitY_neigh

In [ ]:
yhat_SL_neigh=predict(fitY_neigh, newdata = data.frame(test_neigh), onlySL = T)$pred


In [ ]:
with(test_neigh,mean(abs(logprice-yhat_SL_neigh)))

In [ ]:
with(test ,mean(abs(logprice-yhat_Sup)))